In [3]:
from tabula import read_pdf

In [4]:
import pandas as pd

In [5]:
pdf = read_pdf("Doc. 1 - QGC Consolidado_vSent.pdf",pages='all',multiple_tables=True)

Got stderr: Jan 30, 2023 4:08:26 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Jan 30, 2023 4:08:26 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jan 30, 2023 4:08:26 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jan 30, 2023 4:08:26 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Jan 30, 2023 4:08:26 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [192]:
df = []
for i,data in enumerate(pdf):
    if not i == 0 and not i == 1:
        data = pdf[i]
        data = data[data.iloc[:, 0].notna()]
        data = data[data.iloc[:, 1].notna()]
        if data.iloc[:,2].isna().all():
            data = data.drop(data.columns[2], axis=1)
        data = data[data.columns[:8]]
        data.columns = ["ID", "Nome ou Razão Social", "CNPJ/CPF", "Moeda", "Valor do Crédito em Moeda Original","Valor do Crédito em BRL", "Classificação do Crédito", "Endereço"]
        df.append(data)

In [194]:
df = pd.concat(df)

In [198]:
import requests

In [272]:
import re
def validar_cnpj(cnpj):
    """
    Valida CNPJs, retornando apenas a string de números válida.

    # CNPJs errados
    >>> validar_cnpj('abcdefghijklmn')
    False
    >>> validar_cnpj('123')
    False
    >>> validar_cnpj('')
    False
    >>> validar_cnpj(None)
    False
    >>> validar_cnpj('12345678901234')
    False
    >>> validar_cnpj('11222333000100')
    False

    # CNPJs corretos
    >>> validar_cnpj('11222333000181')
    '11222333000181'
    >>> validar_cnpj('11.222.333/0001-81')
    '11222333000181'
    >>> validar_cnpj('  11 222 333 0001 81  ')
    '11222333000181'
    """
    cnpj = ''.join(re.findall('\d', str(cnpj)))

    if (not cnpj) or (len(cnpj) < 14):
        return False

    # Pega apenas os 12 primeiros dígitos do CNPJ e gera os 2 dígitos que faltam
    inteiros = list(map(int, cnpj))
    novo = inteiros[:12]

    prod = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    while len(novo) < 14:
        r = sum([x*y for (x, y) in zip(novo, prod)]) % 11
        if r > 1:
            f = 11 - r
        else:
            f = 0
        novo.append(f)
        prod.insert(0, 6)

    # Se o número gerado coincidir com o número original, é válido
    if novo == inteiros:
        return cnpj
    return False

In [560]:
# codigo para buscar os CNPJs -  eles ja estão nos arquivos
# for i in range(len(df)):
#     print(f"Tentando pegar o dado de {df.iloc[i]['Nome ou Razão Social']} - {df.iloc[i]['ID']}")
#     cnpj = validar_cnpj(df.iloc[i]["CNPJ/CPF"])
#     if cnpj:
#         url = f"https://publica.cnpj.ws/cnpj/{cnpj}"
#         resp = requests.get(url)
#         if resp.status_code == 200:
#             json_resp = resp.json()
#             cnpj_data.append(json_resp)
#             print(json_resp)
#         else:
#             print("CNPJ inválido")
#         time.sleep(20)    

In [441]:
final_1 = pd.read_json('work/data.json')
final_2 = pd.read_json('work/data2.json')
final_3 = pd.read_json('work/data3.json')
final_4 = pd.read_json('work/data4.json')
final_5 = pd.DataFrame(cnpj_data) #aqui é o 5


In [459]:
final = pd.concat([final_1, final_2, final_3, final_4, final_5])

In [460]:
final = final.reset_index(drop=True)

In [462]:
simples = pd.json_normalize(final["simples"])
qualificacao_do_responsavel = pd.json_normalize(final["qualificacao_do_responsavel"])
porte = pd.json_normalize(final["porte"])
estabelecimentos = pd.json_normalize(final["estabelecimento"])


In [528]:
resultado = pd.concat(
    [final, simples, estabelecimentos, qualificacao_do_responsavel, porte], axis="columns"
)

In [529]:
consolidado = pd.merge(df, resultado, left_on="CNPJ/CPF", right_on='cnpj', how="left")

In [533]:
consolidado = consolidado.drop_duplicates(subset=['CNPJ/CPF'])

In [534]:
import numpy as np

In [541]:
consolidado["Valor do Crédito em Moeda Original"] = consolidado["Valor do Crédito em Moeda Original"].astype(str).str.replace(",", "").replace("-", np.nan).astype(float)

In [551]:
consolidado.to_csv("consolidado.csv", sep=";", index=False, decimal=",")